In [1]:
# 采集1min kline data

In [2]:
import pandas as pd
import numpy as np
import jqfactor_analyzer as ja
import numba
import yfinance as yf
import backtrader as bk
import akshare as ak
import os
import scipy
import statsmodels.api as sm
import random
import math

In [3]:
import math
s1 = ['SC0','PG0','AU0','AG0','CU0','PB0','NI0','BC0','A0','B0','M0','JM0','J0','C0','CS0','MA0','PP0','CU0','RU0','Y0','P0',"AU0", "AG0", "CU0", "AL0", "ZN0", "PB0", "NI0", "SN0", "RB0", "HC0",
    "BU0", "RU0", "FU0", "SP0", "WR0", "SS0", "SA0", "UR0", "SR0", "CF0",
    "TA0", "MA0", "FG0", "RS0", "OI0",'AU0','AG0'
    "JR0", "SF0", "SM0", "AP0", "CJ0", "CY0", "RR0", "BB0", "FB0",
    "EB0", "V0", "L0", "PP0", "CS0", "JD0", "BU0", "HC0"]
#s2 = ['SC0','PG0','AU0','AG0','CU0','PB0','NI0','BC0','A0','B0','M0','JM0','J0','C0','CS0','MA0','PP0','CU0','RU0','Y0','P0']

#内盘
start_date = '2021-01-01'
end_date = '2023-08-10'

#@numba.jit(nopython=True)
def future_AA_arb():
    for i in range(1000000):
        futures_foreign_hist_df = []
        futures_zh_daily_sina_df = []
        # bc-cu,jm-j,rb-hc,cf-cy(x),wr-rb
        if i % 100 == 1:
            print(i)
        s11 = random.choice(s1)
        if s11 != 'C0':
            continue
        try:
            futures_foreign_hist_df = ak.futures_zh_daily_sina(symbol=s11)
        except:
            continue
        futures_foreign_hist_df = futures_foreign_hist_df[(futures_foreign_hist_df.date >= start_date) & (futures_foreign_hist_df.date <= '2023-08-10')]
        date_range = pd.date_range(start=futures_foreign_hist_df["date"].min(), end=futures_foreign_hist_df["date"].max(), freq="D")  # freq="D"表示按天，可以按分钟，月，季度，年等
        futures_foreign_hist_df['date'] = pd.to_datetime(futures_foreign_hist_df['date'])
        futures_foreign_hist_df = futures_foreign_hist_df.set_index("date")
        futures_foreign_hist_df = futures_foreign_hist_df.resample("D").ffill()
        #print(futures_foreign_hist_df['close'])
        #print(futures_foreign_hist_df)
    
        #
        s22 = random.choice(s1)
        if s22 == s11:
            continue
        try:
            futures_zh_daily_sina_df = ak.futures_zh_daily_sina(symbol=s22)
        except:
            continue
        futures_zh_daily_sina_df = futures_zh_daily_sina_df[(futures_zh_daily_sina_df.date>=start_date) & (futures_zh_daily_sina_df.date <= '2023-08-10')]
        date_range = pd.date_range(start=futures_zh_daily_sina_df["date"].min(), end=futures_zh_daily_sina_df["date"].max(), freq="D")  # freq="D"表示按天，可以按分钟，月，季度，年等
        futures_zh_daily_sina_df['date'] = pd.to_datetime(futures_zh_daily_sina_df['date'])
        futures_zh_daily_sina_df = futures_zh_daily_sina_df.set_index("date")
        futures_zh_daily_sina_df = futures_zh_daily_sina_df.resample("D").ffill()
        
        #print(futures_zh_daily_sina_df)
        # print(s1[i])
        # print(len(futures_foreign_hist_df))
        # print(len(futures_zh_daily_sina_df))
        #print(np.array(futures_foreign_hist_df['close']))
        ## 计算Pearson相关系数

        #shift 1
        df_for = [futures_foreign_hist_df['close'][0] for i in range(1)] + futures_foreign_hist_df['close'][:-1].tolist()
        
        # pearson_corr, _ = scipy.stats.pearsonr(futures_zh_daily_sina_df['close'][10:], df_for[10:])
        # print("shifted %s-%s Pearson Correlation:" %(s1[i],s2[i]), pearson_corr)
        try:
            pearson_corr, _ = scipy.stats.pearsonr(futures_zh_daily_sina_df['close'], futures_foreign_hist_df['close'])
        except:
            continue
        if abs(pearson_corr) > 0.5:
            print("%s-%s Pearson Correlation:" %(s11,s22), pearson_corr)
                 # 进行ADF检验
            try:
                # result = sm.tsa.coint(futures_zh_daily_sina_df['close'], futures_foreign_hist_df['close'])
                # cointegration_statistic = result[0]
                # p_value = result[1]
                
                # print("cointegration_statistic:", cointegration_statistic)
                # print("p-value:", p_value,'\n')
                print("sgima",np.std(futures_foreign_hist_df['close']-futures_zh_daily_sina_df['close']))
            except:
                continue
#future_AA_arb()

In [8]:
t = [2000,2100,2200,2300]
start_year = 1
end_year = 12
year_strings = []
for i in t:
    for year in range(start_year, end_year + 1):
        year_strings.append(str(i+year))

print(year_strings)

symbols = ['SC0','PG0','AU0','AG0','CU0','PB0','NI0','BC0','A0','B0','M0','JM0','J0','C0','CS0','MA0','PP0','CU0','RU0','Y0','P0',"AU0", "AG0", "CU0", "AL0", "ZN0", "PB0", "NI0", "SN0", "RB0", "HC0",
    "BU0", "RU0", "FU0", "SP0", "WR0", "SS0", "SA0", "UR0", "SR0", "CF0",
    "TA0", "MA0", "FG0", "RS0", "OI0",'AU0','AG0'
    "JR0", "SF0", "SM0", "AP0", "CJ0", "CY0", "RR0", "BB0", "FB0",
    "EB0", "V0", "L0", "PP0", "CS0", "JD0", "BU0", "HC0"]

symbols = [s[:-1] for s in symbols]
for s in symbols:
    df = []
    print(s)
    for i in year_strings:
        csv_filename = './'+s+'_1min.csv'
        try:
            futures_zh_minute_sina_df = ak.futures_zh_minute_sina(symbol=s+i, period="1")
            futures_zh_minute_sina_df.to_csv(csv_filename, mode='a')
        except Exception as e:
            print(e)
            continue
    # #
    # df = pd.read_csv(csv_filename)
    # df.loc[:, 'open':'hold'] = df.loc[:, 'open':'hold'].apply(pd.to_numeric, errors='coerce')
    # df = df.dropna()
    # #df.loc[:,'open':'close'] = df.loc[:,'open': 'close'].astype(float)
    
    # # df["(t+1)-(t)"] = df['close'] - df['close'].shift(-1)      # 2020-11-27修正
    # df["(t+1)-(t)"] = df['close'].shift(-10) - df['close']
    # df = df.dropna()
    # #print(df)
    # df['label'] = 0
    # # 构建对应表
    # df = df.dropna()
    # df['label'] = df["(t+1)-(t)"].apply(lambda x: 1 if x > 0 else 0)
    # df = df.dropna()
    # #df['datetime'] = pd.to_datetime(df['datetime']).astype(int)
    # df.to_csv(csv_filename,mode='w')

['2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2101', '2102', '2103', '2104', '2105', '2106', '2107', '2108', '2109', '2110', '2111', '2112', '2201', '2202', '2203', '2204', '2205', '2206', '2207', '2208', '2209', '2210', '2211', '2212', '2301', '2302', '2303', '2304', '2305', '2306', '2307', '2308', '2309', '2310', '2311', '2312']
SC
Length mismatch: Expected axis has 0 elements, new values have 7 elements
Length mismatch: Expected axis has 0 elements, new values have 7 elements
Length mismatch: Expected axis has 0 elements, new values have 7 elements
PG
Length mismatch: Expected axis has 0 elements, new values have 7 elements
Length mismatch: Expected axis has 0 elements, new values have 7 elements
Length mismatch: Expected axis has 0 elements, new values have 7 elements
Length mismatch: Expected axis has 0 elements, new values have 7 elements
Length mismatch: Expected axis has 0 elements, new values have 7 elements
Length mismatch: 

In [5]:
start = 2020
end = 2313

integer_sequence = range(start, end)
string_sequence = [str(number) for number in integer_sequence]

print(string_sequence)


['2020', '2021', '2022', '2023', '2024', '2025', '2026', '2027', '2028', '2029', '2030', '2031', '2032', '2033', '2034', '2035', '2036', '2037', '2038', '2039', '2040', '2041', '2042', '2043', '2044', '2045', '2046', '2047', '2048', '2049', '2050', '2051', '2052', '2053', '2054', '2055', '2056', '2057', '2058', '2059', '2060', '2061', '2062', '2063', '2064', '2065', '2066', '2067', '2068', '2069', '2070', '2071', '2072', '2073', '2074', '2075', '2076', '2077', '2078', '2079', '2080', '2081', '2082', '2083', '2084', '2085', '2086', '2087', '2088', '2089', '2090', '2091', '2092', '2093', '2094', '2095', '2096', '2097', '2098', '2099', '2100', '2101', '2102', '2103', '2104', '2105', '2106', '2107', '2108', '2109', '2110', '2111', '2112', '2113', '2114', '2115', '2116', '2117', '2118', '2119', '2120', '2121', '2122', '2123', '2124', '2125', '2126', '2127', '2128', '2129', '2130', '2131', '2132', '2133', '2134', '2135', '2136', '2137', '2138', '2139', '2140', '2141', '2142', '2143', '2144',